In [5]:
from fnmatch import fnmatch
from PIL import Image
from PIL import ImageFile
from pathlib import Path
from shutil import copyfile

import imagehash
import mysql.connector
import os
import re

In [6]:
# GENERAL CONFIGS

ImageFile.LOAD_TRUNCATED_IMAGES = True
Image.MAX_IMAGE_PIXELS = None

# == Database config ==
conn = mysql.connector.connect(
  host="localhost",
  user="root",
  password="root"
)

cursor = conn.cursor(buffered=True)

DB_NAME = "blp_dataset"
TABLE_NAME = "image_hashes"

cursor.execute("USE {}".format(DB_NAME))

In [15]:
table_schema = (
    "CREATE TABLE IF NOT EXISTS image_hashes ("
    "  file TEXT NOT NULL,"
    "  hash_file VARCHAR(255) PRIMARY KEY NOT NULL,"
    "  market VARCHAR(45) NOT NULL DEFAULT 'no'"
    ") ENGINE=InnoDB")

cursor.execute(table_schema)

cursor.execute("DROP TABLE IF EXISTS repeated_image_hashes")

table_schema = (
    "CREATE TABLE IF NOT EXISTS repeated_image_hashes ("
    "  file VARCHAR(255) NOT NULL,"
    "  hash_file TEXT NOT NULL"
    ") ENGINE=InnoDB")

cursor.execute(table_schema)

In [16]:
## CLEAR REPEATED IMAGES

# == Files config ==
MAIN_FOLDER = r"D:\images\_myntra"

folder = Path(MAIN_FOLDER)
repeated_files = []


for image in os.listdir(folder):
    image_path = Path(f"{folder}\{image}")

    try:
        img_hash = imagehash.phash(Image.open(image_path))
        image_name = os.path.splitext(image)[0]
        
        cursor.execute("SELECT COUNT(file) FROM image_hashes WHERE hash_file = %s", (str(img_hash),))
        has_hash = cursor.fetchone()

        if has_hash[0] == 0:
            query = "INSERT INTO image_hashes (file, hash_file, market) VALUES (%s, %s, %s)"
            values = (image_name, str(img_hash), "nucleus")

            cursor.execute(query, values)
            conn.commit()
        else:
            query = "INSERT INTO repeated_image_hashes (file, hash_file) VALUES (%s, %s)"
            values = (image_name, str(img_hash))

            cursor.execute(query, values)
            conn.commit()
    except:
        query = "INSERT INTO repeated_image_hashes (file, hash_file) VALUES (%s, %s)"
        values = (image_name, image_name)

        cursor.execute(query, values)
        conn.commit()



In [18]:
# REMOVE IMAGES IN REPEATED TABLE
cursor.execute("SELECT * FROM repeated_image_hashes")
repeated = cursor.fetchall()

MAIN_FOLDER = r"D:\images\_myntra"
folder = Path(MAIN_FOLDER)

for res in repeated:
    image_path = Path(f"{folder}\{res[0]}.jpeg")
    
    try:
        os.remove(image_path)
    except:
        print(image_path)

D:\images\_myntra\10174.jpeg
D:\images\_myntra\10252.jpeg
D:\images\_myntra\10267.jpeg
D:\images\_myntra\10292.jpeg
D:\images\_myntra\10294.jpeg
D:\images\_myntra\10306.jpeg
D:\images\_myntra\10323.jpeg
D:\images\_myntra\10379.jpeg
D:\images\_myntra\10405.jpeg
D:\images\_myntra\10425.jpeg
D:\images\_myntra\10433.jpeg
D:\images\_myntra\10447.jpeg
D:\images\_myntra\10462.jpeg
D:\images\_myntra\10467.jpeg
D:\images\_myntra\10469.jpeg
D:\images\_myntra\10487.jpeg
D:\images\_myntra\10502.jpeg
D:\images\_myntra\10565.jpeg
D:\images\_myntra\10614.jpeg
D:\images\_myntra\10624.jpeg
D:\images\_myntra\10657.jpeg
D:\images\_myntra\10679.jpeg
D:\images\_myntra\10699.jpeg
D:\images\_myntra\10710.jpeg
D:\images\_myntra\10713.jpeg
D:\images\_myntra\10730.jpeg
D:\images\_myntra\10732.jpeg
D:\images\_myntra\10751.jpeg
D:\images\_myntra\10754.jpeg
D:\images\_myntra\10755.jpeg
D:\images\_myntra\10757.jpeg
D:\images\_myntra\10813.jpeg
D:\images\_myntra\10844.jpeg
D:\images\_myntra\10855.jpeg
D:\images\_myn

D:\images\_myntra\6943.jpeg
D:\images\_myntra\6951.jpeg
D:\images\_myntra\6953.jpeg
D:\images\_myntra\6960.jpeg
D:\images\_myntra\6963.jpeg
D:\images\_myntra\6965.jpeg
D:\images\_myntra\6976.jpeg
D:\images\_myntra\6984.jpeg
D:\images\_myntra\6985.jpeg
D:\images\_myntra\6990.jpeg
D:\images\_myntra\6995.jpeg
D:\images\_myntra\7000.jpeg
D:\images\_myntra\7003.jpeg
D:\images\_myntra\7008.jpeg
D:\images\_myntra\7015.jpeg
D:\images\_myntra\7017.jpeg
D:\images\_myntra\7026.jpeg
D:\images\_myntra\7028.jpeg
D:\images\_myntra\7031.jpeg
D:\images\_myntra\7035.jpeg
D:\images\_myntra\7046.jpeg
D:\images\_myntra\7052.jpeg
D:\images\_myntra\7063.jpeg
D:\images\_myntra\7064.jpeg
D:\images\_myntra\7078.jpeg
D:\images\_myntra\7083.jpeg
D:\images\_myntra\7085.jpeg
D:\images\_myntra\7086.jpeg
D:\images\_myntra\7089.jpeg
D:\images\_myntra\7093.jpeg
D:\images\_myntra\710.jpeg
D:\images\_myntra\7100.jpeg
D:\images\_myntra\7102.jpeg
D:\images\_myntra\7104.jpeg
D:\images\_myntra\7110.jpeg
D:\images\_myntra\711

In [10]:
## CLEAR REPEATED IMAGES IN DIFFERENT FOLDERS

# == Files config ==
MAIN_FOLDER = r"D:\images\_haven"

folder = Path(MAIN_FOLDER)
repeated_files = []


for image in os.listdir(folder):
    image_path = Path(f"{folder}\{image}")


    img_hash = imagehash.phash(Image.open(image_path))
    image_name = os.path.splitext(image)[0]

    cursor.execute("SELECT COUNT(file) FROM image_hashes WHERE hash_file = %s", (str(img_hash),))
    has_hash = cursor.fetchone()
    
    if has_hash[0] == 0:
        print("Safe")
    else:
        query = "INSERT INTO repeated_image_hashes (file, hash_file) VALUES (%s, %s)"
        values = (image_name, str(img_hash))

        cursor.execute(query, values)
        conn.commit()

FileNotFoundError: [WinError 3] O sistema não pode encontrar o caminho especificado: 'D:\\images\\_haven'

In [5]:
# Get list of numbers in sequence that don't have files

count = 0
nums = []
MAIN_FOLDER = r"D:\images\illegal"

while (len(nums) < 174 or count < 58771):
    img = Path(f"{MAIN_FOLDER}\{count}.jpeg")
    if not img.exists():
        nums.append(count)

    count = count + 1


In [6]:
# Rename images based on list of numbers that don't exist into illegal folder

folder = Path("D:\images\_temp")
folder_tmp = Path("D:\images\_folder_tmp")
cont = 1

for image in os.listdir(folder):
    image_path = Path(f"{folder}\{image}")

    try:
        img = Image.open(image_path).convert('RGB').save(f'{folder_tmp}\{nums[cont]}.jpeg')
        cont = cont + 1
    except:
        print(image_name)

In [ ]:
print(nums)